In [1]:
# !pip install ultralytics

In [2]:
import os
from ultralytics import YOLO
import numpy as np
import cv2
import pandas as pd


In [3]:
model_path = "../models/yolov8m-seg.pt"

In [4]:

class YOLOSegmentation:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def detect(self, img):
        height, width, channels = img.shape

        results = self.model.predict(source=img.copy(), save=False, save_txt=False)
        result = results[0]
        segmentation_contours_idx = []
        for seg in result.masks.xyn:
            seg[:, 0] *= width
            seg[:, 1] *= height
            segment = np.array(seg, dtype=np.int32)
            segmentation_contours_idx.append(segment)
        bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        scores = np.array(result.boxes.conf.cpu(), dtype="float").round(2)
        return bboxes, class_ids, segmentation_contours_idx, scores


In [5]:
classes_ids = [
    2, # Car
    7, # Truck
    5, # Bus
]

In [6]:
def extract_contours(folder: str, output_dir: str, Segmentator: YOLOSegmentation, transform_to_drawings=False) -> None:
    for image_name in os.listdir(folder)[:]:
        if not (image_name.endswith(".jpg") or image_name.endswith(".jpeg")):
            continue
        start_path = folder + '/' + image_name
        end_path = f"{output_dir}/"
        os.makedirs(end_path, exist_ok=True)

        img = cv2.imread(start_path)
        if img is None:
            print(Exception(f"Can't read an image: {image_name}"))
            continue
        img = cv2.resize(img, None, fx=0.5, fy=0.5)

        bboxes, classes, segmentations, scores = Segmentator.detect(img)


        # for bbox, class_id, seg, score in zip(bboxes, classes, segmentations, scores):
        #     (x, y, x2, y2) = bbox
        #
        #     cv2.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 2)
        #     cv2.polylines(img, [seg], True, (0, 0, 255), 4)
        #     edges = cv2.putText(img, str(class_id), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        #
        #     image_name_pieces = image_name.split(".")
        #     image_name = image_name_pieces[0] + "_processed" + "." + image_name_pieces[-1]
        #     cv2.imwrite(end_path + "/" + image_name, edges)
        # continue


        for item, object_class in enumerate(classes):
            if object_class not in classes_ids:
                continue
            else:
                break


        points = np.array(segmentations[item])

        mask = np.zeros(img.shape[:2], dtype=np.uint8)

        cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)

        img = cv2.bitwise_and(img, img, mask=mask)

        if transform_to_drawings:

            blurred_image = cv2.GaussianBlur(img.copy(),(5,5),0)

            img = cv2.Canny(blurred_image, 100, 160)

        cv2.imwrite(end_path + "/" + image_name, img)


In [7]:
WHOLE_DATA_PATH = "../output"
RESULT_PATH = "../extracted_cars"

In [ ]:
for dataset in ['train', 'test']:
    folder = WHOLE_DATA_PATH + "/" + dataset
    result_folder = RESULT_PATH + "/" + dataset

    extract_contours(folder=folder,
                     output_dir=result_folder,
                     Segmentator=YOLOSegmentation(model_path),
                     transform_to_drawings=False)



0: 640x640 1 car, 2307.5ms
Speed: 78.1ms preprocess, 2307.5ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 2121.4ms
Speed: 0.0ms preprocess, 2121.4ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 truck, 2211.0ms
Speed: 31.3ms preprocess, 2211.0ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 2219.8ms
Speed: 0.0ms preprocess, 2219.8ms inference, 31.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bus, 2121.6ms
Speed: 15.6ms preprocess, 2121.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 2038.5ms
Speed: 0.0ms preprocess, 2038.5ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 2040.3ms
Speed: 0.0ms preprocess, 2040.3ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 2065.7ms
Speed: 15.6ms preprocess, 2065.7ms inference, 0.0ms postprocess per image